In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arketing-campaign/marketing_campaign.csv
/kaggle/input/arketing-campaign/marketing_campaign.xlsx


In [2]:
from datetime import datetime

In [3]:
RAW_PATH = '/kaggle/input/arketing-campaign/marketing_campaign.csv'
OUT_DIR = '/kaggle/working/processed'
FULL_OUT_CSV = os.path.join(OUT_DIR, 'processed.csv')
FULL_OUT_PKL = os.path.join(OUT_DIR, 'processed.pkl')
HYP_OUT_CSV = os.path.join(OUT_DIR, 'hypothesis_ready.csv')

os.makedirs(OUT_DIR, exist_ok=True)

<h2> 1. Load & Check the Data </h2>

In [4]:
df = pd.read_csv(RAW_PATH, sep=';')
df.shape

(2240, 29)

In [5]:
df.head(5)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0


In [6]:
df.isna().sum().sort_values(ascending=False).head()

Income               24
ID                    0
NumDealsPurchases     0
Z_Revenue             0
Z_CostContact         0
dtype: int64

<h2> 2. Process Missing Data</h2>

In [7]:
# Missing Income Data to Median
income_median = df['Income'].median(skipna=True)
df['Income'] = df['Income'].fillna(income_median)
df.isna().sum().sort_values(ascending=False).head()

ID                   0
NumDealsPurchases    0
Z_Revenue            0
Z_CostContact        0
Complain             0
dtype: int64

In [8]:
# Delete if year_birth < 1925 (age higher than 100) OR age lower than 10
df = df[df['Year_Birth'].between(1925, datetime.now().year - 10)]
df['Year_Birth'].sort_values().head()

1950    1940
424     1941
894     1943
1150    1943
39      1943
Name: Year_Birth, dtype: int64

In [9]:
df.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')

In [10]:
CHANNEL_COLS = [
    "NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases",
    "NumWebVisitsMonth", "NumDealsPurchases"
]
for c in CHANNEL_COLS:
    if c in df.columns:
        df[c] = df[c].fillna(0)

In [11]:
df.isna().sum().sort_values(ascending=False).head()

ID                   0
NumDealsPurchases    0
Z_Revenue            0
Z_CostContact        0
Complain             0
dtype: int64

In [12]:
core_cols = ['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
             'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
             'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
             'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
             'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
             'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
             'AcceptedCmp2', 'Complain', 'Response']

In [13]:
df = df.dropna(subset=[c for c in core_cols if c in df.columns])

<h2>3. Remove Income Outliers</h2>
<h3>: Used IQR Method</h3>

In [14]:
Q1 = df['Income'].quantile(0.25)
Q3 = df['Income'].quantile(0.75)
IQR = Q3 - Q1
upper_fence = Q3 + 1.5*IQR
shape_before = df.shape
df = df[df['Income'] <= upper_fence]
shape_after = df.shape
print(f'Shape -> before: {shape_before}, after: {shape_after}')
print(f'removed {shape_before[0] - shape_after[0]} outliers')

Shape -> before: (2237, 29), after: (2229, 29)
removed 8 outliers


<h2>4. Include "Age" and "TotalSpend"</h2>

In [15]:
REFERENCE_YEAR = 2025
df['Age'] = REFERENCE_YEAR - df['Year_Birth']
spend_cols = ['MntWines', 'MntFruits', 'MntMeatProducts', 
              'MntFishProducts', 'MntSweetProducts','MntGoldProds']
df['TotalSpend'] = df[spend_cols].sum(axis=1)

In [16]:
df.head(5)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Age,TotalSpend
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,0,0,0,0,0,3,11,1,68,1617
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,0,0,0,0,0,3,11,0,71,27
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,0,0,0,0,0,3,11,0,60,776
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,0,0,0,0,0,3,11,0,41,53
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,0,0,0,0,0,3,11,0,44,422


<h2>5. Process Response to Campaigns</h2>

In [17]:
# Make all Responses to 1/0
true_like = {1, "1", True, "True", "TRUE", "Yes", "YES", "Y"}
df['Response'] = df['Response'].apply(lambda x: 1 if x in true_like else 0).astype(int)

In [18]:
accepted_cols = [c for c in ["AcceptedCmp1","AcceptedCmp2","AcceptedCmp3","AcceptedCmp4","AcceptedCmp5"] if c in df.columns]
if accepted_cols:
    df['AcceptedCnt'] = df[accepted_cols].sum(axis=1).astype(int)

In [19]:
df.head(5)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Age,TotalSpend,AcceptedCnt
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,0,0,0,0,3,11,1,68,1617,0
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,0,0,0,0,3,11,0,71,27,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,0,0,0,0,3,11,0,60,776,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,0,0,0,0,3,11,0,41,53,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,0,0,0,0,3,11,0,44,422,0


<h2>6. Save the Processed Data </h2>

In [20]:
final_cols= ['ID', 'Year_Birth', 'Age', 'Education', 'Marital_Status', 'Income', 'Kidhome',
             'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
             'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
             'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
             'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
             'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
             'AcceptedCmp2', 'Complain', 'Response', 'TotalSpend', 'AcceptedCnt']

In [21]:
hypo_df = df[final_cols].copy()

In [22]:
df.to_csv(FULL_OUT_CSV, index=False)
df.to_pickle(FULL_OUT_PKL)
hypo_df.to_csv(HYP_OUT_CSV, index=False)
print("Saved:")
print(" -", FULL_OUT_CSV)
print(" -", FULL_OUT_PKL)
print(" -", HYP_OUT_CSV)

Saved:
 - /kaggle/working/processed/processed.csv
 - /kaggle/working/processed/processed.pkl
 - /kaggle/working/processed/hypothesis_ready.csv


In [23]:
variable_doc = {
    "ID": "User Id",
    "Year_Birth": "Year of birth (full data)",
    "Age": "2025 - Year_Birth",
    "Education": "Education",
    "Marital_Status": "Marital_Status",
    "Kidhome": "Number of Kids",
    "Teenhome": "Number of Teenagers",
    "Income": "Annual Income (Upper fence by IQR method, FIlled None with Median",
    "Recency": "Dates passed after the last purchase",
    "MntWines ~ MntGoldProds": "Categorical Spend",
    "TotalSpend": "TotalSpend",
    "AcceptedCmp1~5": "Previous Campaigns (accepted:1, not:0)",
    "Response": "Response to the last campain (1/0)",
    "AcceptedCnt": "Total accepted campaigns",
    "NumWebPurchases": "Number of Web Purchases",
    "NumCatalogPurchases": "Number of Catalog Purchases",
    "NumStorePurchases": "Number of Store Purchases",
    "NumWebVisitsMonth": "Number of Monthly Web Visits",
    "NumDealsPurchases": "Number of Deals Purchases"
}

pd.Series(variable_doc).to_frame("Description")

,Description
ID,User Id
Year_Birth,Year of birth (full data)
Age,2025 - Year_Birth
Education,Education
Marital_Status,Marital_Status
Kidhome,Number of Kids
Teenhome,Number of Teenagers
Income,"Annual Income (Upper fence by IQR method, FIll..."
Recency,Dates passed after the last purchase
MntWines ~ MntGoldProds,Categorical Spend
